# LangChain Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM

Messages are objects that contain:
- **Role** - Identifies the message tpye (e.g. **system**, **user**)
- **Content** - Represents the actual content of the message (like text, images, audio, documents, etc.)
- **Metadata**: Optional fields such as response information, message IDs, and token usage.

## Load env variables

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## Basic usage

In [3]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("gpt-4")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

# Use with chat models
messages = [system_msg, human_msg]
model.invoke(messages)

AIMessage(content="Hello! As an artificial intelligence, I don't have feelings, but I'm here and ready to assist you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 23, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CjR5Yk2q8mQMnOdEz8Z5Fy4eCEdej', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d399088e-ea66-4b85-84e5-351991fc3bec-0', usage_metadata={'input_tokens': 23, 'output_tokens': 30, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Text prompts
Text prompts are strings - ideal for straightforward generation tasks where you don't need to retain conversation history.

In [ ]:
response = model.invoke("Write a haiku about spring")

### Message prompts
Alternatively, you can pass in a list of messages to the model by providing a list of message objects.

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage("You are a poetry expert"),
    HumanMessage("Write a haiku about spring"),
    AIMessage("Cherry blossoms bloom...")
]

response = model.invoke(messages)

### Dictionary format
You can also specify messages firectly in OpenAI chat completions format.

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

messages = [
    {"role": "system", "content": "You are a poetry expert"},
    {"role": "user", "content": "Write a haiku about spring"},
    {"role": "assistant", "content": "Cherry blossoms bloom..."}
]

response = model.invoke(messages)

## Message types

### SystemMessage
A SystemMessage represent an initial set of instructions that primes the model's behavior. You can use a system message to set the tone, define the model's role, and establish guidelines for responses

In [ ]:
system_msg = SystemMessage("You are a helpful coding assistant.")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]

response = model.invoke(messages)

### HumanMessage
A HumanMessage represents user input and interactions. They can contain text, images, audio, files, and any other amount of multimodal content.

In [ ]:
# Message object
response = model.invoke([
    HummanMessage("What is machine learning?")
])

# String shortcut
response = model.invoke("What is machine learning?")

### AIMessage
An AIMessage represents the output of a model invocation. They can include multimodal data, tool calls, and provider-specific meta data that you can later access.

In [5]:
response = model.invoke("Explain AI")
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


**AIMessage** objects are returnes by the model when calling it, which contains all of the associated metadata in the response.

Providers weigh/contextualize types fo messages differently, which means it's sometimes helpful to manually create a new AIMessage object and insert it into the message history as if it came from the model.

In [8]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# Create an AI message manually (e.g., for conversation history)
ai_msg = AIMessage("I'd be happy to help you with that question!")

# Add to conversation history
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg, # Insert as if it came from the model
    HumanMessage("Great! What's 2 + 2?")
]

reponse = model.invoke(messages)

## Message content
You can think of a message's content as the payload of data that gets sent to the model. Messages have a **content** attribute that is loosely-typed, supporting strings and lists of untyped objects (e.g. dictionaries). This allows support for provider-native structures directly in LangChain chat models, such as multimodal content and other data.

LangChain chat models accept message content in the `content` attribute, this may contain either:
- A string
- A list of content blocks in a provider-native format
- a iist of LangChain's standard content blocks

In [14]:
from langchain.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g. OpenAI)
human_message = HumanMessage(content = [
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": "https://example.com/image.jpg"}
])

# List of standard content blocks
human_message = HumanMessage(content_block = [
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"}
])

human_message

ValidationError: 2 validation errors for HumanMessage
content.str
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type